In [1]:
import zipfile
import os
import pandas as pd
import numpy as np
import folium
from fastkml import kml
import geopandas as gpd
from general_fun import *

In [2]:
engine, conn = connect_database()

2023-11-10 13:48:59,571 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-11-10 13:48:59,572 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-10 13:48:59,586 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-11-10 13:48:59,587 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-10 13:48:59,588 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-11-10 13:48:59,588 INFO sqlalchemy.engine.Engine [raw sql] ()


In [3]:
# read KML content 
kmz = zipfile.ZipFile(os.path.join(os.getcwd(),"gis","Werken.kmz"), 'r')  # extract zip file first, then read kmz file inside the extracted folder
kml_content = kmz.open('doc.kml', 'r').read()  # kml content

# create KML object
k = kml.KML()
k.from_string(kml_content)

# read features from docs to folders to records and then extract geometries - in my case, Shapely points
docs = list(k.features())
folders = []
for d in docs:
    folders.extend(list(d.features()))

records = []
for f in folders:
    records.extend(list(f.features()))

geoms = [element.geometry for element in records]
names = [element.name for element in records]
descriptions = [element.description for element in records]

In [4]:
projects = pd.read_excel(os.path.join(os.getcwd(), "gis", "werken.ods"), engine="odf", converters={'date_start':str,'date_end':str, 'project_number':str})

projects['date_start'] = pd.to_datetime(projects['date_start'], format='ISO8601', utc=False)
projects['date_end'] = pd.to_datetime(projects['date_end'], format='ISO8601', utc=False)

projects["id"] = projects.index + 1

In [5]:
projects.head(50)

,name,company,year,date_start,date_end,days,city,country,description,vessels / office,project_number,id
0,De Watterakkers fase 3,Seignette,2003,2003-01-21,2003-06-03,20.0,Heemskerk,The Netherlands,NaN,NaN,NaN,1
1,Reconstructie Orionweg,Heijmans,2003,2003-08-25,2004-01-09,NaN,Haarlem,The Netherlands,NaN,NaN,612045,2
2,IJSEI,Koop Tjuchem,2005,2005-01-31,2005-07-01,104.0,Amsterdam,The Netherlands,Bouw van een kistdam in het IJ inclusief bouwk...,NaN,650030,3
3,Groot onderhoud A6,Koop Tjuchem,2006,2006-05-25,2006-05-26,2.0,Lelystad,The Netherlands,NaN,NaN,NaN,4
4,Spoor Verbreding Amsterdam Utrecht,Koop Tjuchem,2006,NaT,NaT,NaN,Amsterdam,The Netherlands,NaN,NaN,221151,5
5,Foodstrip,Koop Tjuchem,2006,NaT,NaT,NaN,Amsterdam,The Netherlands,NaN,NaN,251032,6
6,Parkeerterrein Papierweg,Koop Tjuchem,2006,NaT,NaT,NaN,Amsterdam,The Netherlands,NaN,NaN,261019,7
7,Parke Nacional Arikok - Roads & Buildings,MNO Vervat,2007,2007-02-05,2007-08-21,198.0,Santa Cruz,Aruba,De eerste opdracht bestaat uit het inrichten v...,NaN,303-001,8
8,Tijdelijke Busbrug NZ-Lijn,MNO Vervat,2007,NaT,NaT,NaN,Amsterdam,The Netherlands,NaN,NaN,261042,9
9,Strandhoogtemeting,Alkyon Hydraulic Consultancy & Research,2007,NaT,NaT,NaN,Egmond aan Zee,The Netherlands,Strandhoogtemeting tussen strandpaal 36.00 en ...,NaN,NaN,10


In [6]:
df = gpd.read_file(os.path.join(os.getcwd(),"gis", "Werken.shp"))
df.columns = map(str.lower, df.columns)
df.rename(
    columns={
        "descript": "description",
    },
    inplace=True,
)
df["latitude"] = df.apply(
    lambda x: x["geometry"].centroid.coords.xy[1][0], axis=1
)
df["longitude"] = df.apply(
    lambda x: x["geometry"].centroid.coords.xy[0][0], axis=1
)

df["company"] = [x.rsplit(' - ', 1)[-1] for x in df["name"]]
df["name"] = [x.rsplit(' - ', 1)[0] for x in df["name"]]

company_colors = {'Alkyon Hydraulic Consultancy & Research':'green','Heijmans':'darkred','Koop Tjuchem':'lightgray','MNO Vervat':'orange','Seignette':'lightblue','Van Oord':'darkblue'}

df["company_color"] = df["company"].map(company_colors)

geoms = [[point.xy[1][0], point.xy[0][0]] for point in df.geometry]

In [7]:
df.head(35)

,name,description,icon,elevation,geometry,latitude,longitude,company,company_color
0,Deepening and widening shipping channel of Kla...,None,information,0,POINT (21.13568 55.68474),55.684738,21.135683,Van Oord,darkblue
1,Strandhoogtemeting,None,information,0,POINT (4.61951 52.61906),52.619056,4.619506,Alkyon Hydraulic Consultancy & Research,green
2,Groot onderhoud A6,None,information,0,POINT (5.42397 52.43434),52.434345,5.423969,Koop Tjuchem,lightgray
3,IJSEI,None,information,0,POINT (4.90095 52.37981),52.379805,4.900949,Koop Tjuchem,lightgray
4,Tijdelijke Busbrug NZ-Lijn,None,information,0,POINT (4.90115 52.37722),52.377217,4.901152,MNO Vervat,orange
5,Spoor Verbreding Amsterdam Utrecht,None,information,0,POINT (4.95906 52.29840),52.298397,4.959057,Koop Tjuchem,lightgray
6,Foodstrip,None,information,0,POINT (4.94738 52.29274),52.292741,4.947381,Koop Tjuchem,lightgray
7,Parkeerterrein Papierweg,None,information,0,POINT (4.86676 52.40248),52.402475,4.866759,Koop Tjuchem,lightgray
8,KKA Cargill,None,information,0,POINT (4.86773 52.40209),52.402094,4.867735,MNO Vervat,orange
9,Tunnelwerk 2008,None,information,0,POINT (4.92465 52.33419),52.334195,4.924651,MNO Vervat,orange


In [8]:
sql = """select * from home.house"""
house = pd.read_sql(sql, conn)

# house = pd.read_pickle("house.pkl")
geometry = gpd.points_from_xy(house["longitude"], house["latitude"])
geo_house = gpd.GeoDataFrame(house, geometry=geometry)
geo_house_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_house.geometry]

2023-11-10 13:48:59,839 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-10 13:48:59,840 INFO sqlalchemy.engine.Engine DESCRIBE `home`.`select * from home.house`
2023-11-10 13:48:59,841 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-10 13:48:59,865 INFO sqlalchemy.engine.Engine select * from home.house
2023-11-10 13:48:59,866 INFO sqlalchemy.engine.Engine [raw sql] ()


In [9]:
mapit = folium.Map(location=np.array(house.loc[house["location"].isin(["Boskoop"]), ["latitude", "longitude"]])[0], tiles="OpenStreetMap", zoom_start=9)

for jj, coordinates in enumerate(geo_house_list):
    mapit.add_child(folium.Marker(location=coordinates, popup=house["location"][jj], icon=folium.Icon(color='red', icon='home')))

for jj, geom in enumerate(geoms):
    mapit.add_child(folium.Marker(location=(df["geometry"][jj].y, df["geometry"][jj].x), popup=f"<b>{df['name'][jj]}</b>\n\n{df['description'][jj]}", icon=folium.Icon(color=df["company_color"][jj], icon='person-digging', prefix='fa')))


mapit.save(os.path.join(os.getcwd(),"html", "projects.html"))
mapit